# Offline Path Planning Optimization by Segmented Bezier Curves

This notebook demonstrates a practical example of path planning using Bezier curves in each segment. In both explored approaches, a single Bezier curve is fit within each segment. The control points are then determined to obtain a path of minimum curvature, curvature variation with curvature continuity.

In [447]:
%matplotlib widget
import time
import nlopt
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as optimize
import scipy.special
from optimparallel import minimize_parallel
from cubic_bezier_planner import calc_bezier_path
from cubic_spline_planner import calc_spline_course

In [448]:
class Path:
    def __init__(self, x, y, yaw, k):
        self.x = x
        self.y = y
        self.yaw = yaw
        self.k = k

## Calculating Quintic Bezier Curve Control Points
A quintic Bezier path requires 6 control points. It has the key advantage that the curvature or likewise the second derivative may be manipulated at the start and endpoints to obtain a curvature continuous path. The offset expresses the distance from each endpoint to middle control points as a fraction of the segment length. The method implemented to derive the 6 control points from a position, heading and second derivative vector at the start and endpoints are described in this paper:
http://www2.informatik.uni-freiburg.de/~lau/students/Sprunk2008.pdf

![](figures/quintic_points.png)
![](figures/control_point_calc.png)

In [449]:

#Creating quintic bezier path controlling position, first, second derivative at start and end points
def calc_5ord_bezier_path(sx, sy, sd, sdd, ex, ey, ed, edd, offset_1, offset_2):

    sd = np.multiply(sd, 0.2*offset_1)
    sdd = np.multiply(sdd, 0.05)
    ed = np.multiply(ed, 0.2*offset_2)
    edd = np.multiply(edd, 0.05)

    # control points
    p0 = [sx, sy]
    p1 = np.add(p0, sd)

    p2 = np.multiply(p1, 2)
    p2 = np.add(p2, sdd)
    p2 = np.subtract(p2, p0)

    p5 = [ex, ey]
    p4 = np.subtract(p5, ed)

    p3 = np.multiply(p4, 2)
    p3 = np.add(p3, edd)    
    p3 = np.subtract(p3, p5)

    control_points = np.array((p0, p1, p2, p3, p4, p5))
    path = calc_bezier_path(control_points, n_points=170)

    return path, control_points

# Spline Class
A spline class is initated with an input set of waypoints and a maximum deviation. Boundary lines are constructed, and this class features functions for calculating derivatives, yaw, curvature and path distance.

In [450]:
class Spline():

    def __init__(self, ax, ay, bound):
        
        #input waypoint coordinates
        ayaw, k = self.calc_yaw_curvature(ax, ay)
        dyaw = ayaw.copy()
        for n in range(1, len(ax)-1):
            yaw = 0.5*(dyaw[n] + dyaw[n-1])
            ayaw[n] = yaw
        self.waypoints = Path(ax, ay, ayaw, k)
        
        # defines and sets left and right boundary lines
        self.bound = bound
        lax, lay, rax, ray = self.init_boundary()
        self.left_bound = Path(lax, lay, None, None)
        self.right_bound = Path(rax, ray, None, None)

        # default unoptimized cubic bezier path to initialize curvature
        bx, by, ctr_pt_x, ctr_pt_y = self.quintic_bezier_path(ax, ay, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
        byaw, bk = self.calc_yaw_curvature(bx, by)
        self.default_path = Path(bx, by, byaw, bk)

        # default seeding path for cubic spline
        cx, cy, cyaw, ck, _ = calc_spline_course(ax, ay, 0.5)
        self.seeding_path = Path(cx, cy, cyaw, ck)

        self.optimized_path = Path([], [], [], [])
        self.ctr_points = Path(ctr_pt_x, ctr_pt_y, [], [])

    # Calculates the first derivative of input arrays
    def calc_d(self, x, y):

        dx, dy = [], []

        for i in range(0, len(x)-1):
            dx.append(x[i+1] - x[i])
            dy.append(y[i+1] - y[i])
        
        dx.append(dx[-1])
        dy.append(dy[-1])
        return dx, dy

    # Calculates yaw and curvature given input path
    def calc_yaw_curvature(self, x, y):

        dx, dy = self.calc_d(x,y)
        ddx, ddy = self.calc_d(dx, dy)
        yaw = []
        k = []

        for i in range(0, len(x)):
            yaw.append(math.atan2(dy[i], dx[i]))
            k.append( (ddy[i] * dx[i] - ddx[i] * dy[i]) / ((dx[i]**2 + dy[i]**2)**(3/2)) )
    
        return yaw, k

    # Calculates total distance of the path
    def calc_path_dist(self, x, y):

        dx = np.absolute(self.calc_d(np.zeros(len(x)), x))
        dy = np.absolute(self.calc_d(np.zeros(len(y)), y))
        ddist = np.hypot(dx, dy)

        return np.sum(ddist)


    # Determines position of boundary lines for visualization
    def init_boundary(self):

        rax, ray, lax, lay = [], [], [], []

        for n in range(0, len(self.waypoints.yaw)):
            lax.append(self.waypoints.x[n] - self.bound*np.sin(self.waypoints.yaw[n]))
            lay.append(self.waypoints.y[n] + self.bound*np.cos(self.waypoints.yaw[n]))
            rax.append(self.waypoints.x[n] + self.bound*np.sin(self.waypoints.yaw[n]))
            ray.append(self.waypoints.y[n] - self.bound*np.cos(self.waypoints.yaw[n]))
        
        return lax, lay, rax, ray

## Determining Heading & First Derivative Vector
The heading is formed as the normal to the bisector of the angle formed at each waypoint. The target heading at the start and end waypoint are fixed as is. The first derivative vector is obtained from the target heading and the less of the distances to the next or previous waypoint.
https://users.soe.ucsc.edu/~elkaim/Documents/camera_WCECS2008_IEEE_ICIAR_58.pdf

![](figures/bisector.png)  
![](figures/tan_length.png)  
![](figures/first_der.png)


In [451]:
class Spline(Spline):

    # Approximates the first derivative vector at all waypoints
    def calc_1st_derivative_heuristic(self, ax, ay):

        n_points = len(ax)
        dx, dy = [], []

        for n in range(n_points):

            # linear distance to next and previous waypoint
            if n == 0:
                dist = np.hypot( (ax[n+1] - ax[n]), (ay[n+1] - ay[n]) )
            elif n == n_points-1:
                dist = np.hypot( (ax[n-1] - ax[n]), (ay[n-1] - ay[n]) )
            else: 
                d0 = np.hypot( (ax[n-1] - ax[n]), (ay[n-1] - ay[n]) )
                d1 = np.hypot( (ax[n+1] - ax[n]), (ay[n+1] - ay[n]) )
                dist = min(d0, d1)

            dx.append(dist * np.cos(self.waypoints.yaw[n]))
            dy.append(dist * np.sin(self.waypoints.yaw[n]))

        return dx, dy

## Determining Second Derivative Vector
The heading is formed as the normal to the bisector of the angle formed at each waypoint. The target heading at the start and end waypoint are fixed as is. The first derivative vector is obtained from the target heading and the less of the distances to the next or previous waypoint.  http://www2.informatik.uni-freiburg.de/~lau/students/Sprunk2008.pdf

![](figures/heuristic.png)  
![](figures/2nd_der.png)
![](figures/weighted_avg.png)
![](figures/weights.png)

In [452]:
class Spline(Spline):

    # Approximates the second derivative vector at all waypoints
    def calc_2nd_derivative_heuristic(self, ax, ay, dx, dy):

        n_points = len(ax)
        ddx, ddy = [], []

        for n in range(n_points):

            # for startpoint, start conditions of first bezier curve
            if n == 0:
                b = np.array([ax[n], ay[n]])
                c = np.array([ax[n+1], ay[n+1]])
                tb = np.array([dx[n], dy[n]])
                tc = np.array([dx[n+1], dy[n+1]])

                dd_vect = -6.0*b - 4.0*tb - 2.0*tc + 6.0*c
                dd_vect = [0, 0]

            elif n == n_points - 1:
                a = np.array([ax[n-1], ay[n-1]])
                b = np.array([ax[n], ay[n]])
                ta = np.array([dx[n-1], dy[n-1]])
                tb = np.array([dx[n], dy[n]])

                dd_vect = 6.0*a + 2.0*ta + 4.0*tb - 6.0*b
                dd_vect = [0, 0]



            # for all points in between, take weighted average at discontinuity
            else:
                a = np.array([ax[n-1], ay[n-1]])
                b = np.array([ax[n], ay[n]])
                c = np.array([ax[n+1], ay[n+1]])

                ta = np.array([dx[n-1], dy[n-1]])
                tb = np.array([dx[n], dy[n]])
                tc = np.array([dx[n+1], dy[n+1]])

                # determining distance for calculating weights
                ab = a-b
                bc = b-c
                ab = np.hypot(ab[0], ab[1])
                bc = np.hypot(bc[0], bc[1])
                alpha = bc / (ab + bc)
                beta = ab / (ab + bc)

                dd_vect = alpha*(6.0*a + 2.0*ta + 4.0*tb - 6.0*b) + beta*(-6.0*b - 4.0*tb - 2.0*tc + 6.0*c)

            ddx.append(dd_vect[0])
            ddy.append(dd_vect[1])

        return ddx, ddy

# Quintic Bezier Curves in Segments
A single quintic bezier curve is fit in each segment, with a specified vector specifying tangent length as described
![](figures/cubic_segment.png)

In [453]:
class Spline(Spline):

    # Approximated quintic bezier path with curvature continuity
    def quintic_bezier_path(self, ax, ay, offset):

        dyaw, _ = self.calc_yaw_curvature(ax, ay)
        
        # control point and path array
        cx, cy, ctr_pt_x, ctr_pt_y = [], [], [], []

        # obtain target second derivative
        dx, dy = self.calc_1st_derivative_heuristic(ax, ay)
        ddx, ddy = self.calc_2nd_derivative_heuristic(ax, ay, dx, dy)

        # for n waypoints, there are n-1 bezier curves
        for i in range(len(ax)-1):

            path, points = calc_5ord_bezier_path( ax[i], ay[i], np.array( [dx[i], dy[i]] ), np.array( [ddx[i], ddy[i]] ), 
                                                ax[i+1], ay[i+1], np.array( [dx[i+1], dy[i+1]] ), np.array( [ddx[i+1], ddy[i+1]] ),
                                                offset[i], offset[i+1] )

            cx = np.concatenate((cx, path.T[0][:-1]))
            cy = np.concatenate((cy, path.T[1][:-1]))
            
            for p in points:
                ctr_pt_x.append( p[0] )
                ctr_pt_y.append( p[1] )

        return cx, cy, ctr_pt_x, ctr_pt_y

## Optimization Strategy

The optimization strategy is essentially to minimize curvature and variation in curvature throughout the entirety of the path

![](figures/cost_func.png)

In [454]:
class Spline(Spline):

    # Objective function for quintic bezier
    def quintic_objective_func(self, params):

        ax = self.waypoints.x.copy()
        ay = self.waypoints.y.copy()

        # calculate offsets and input waypoints
        waypoints = len(self.waypoints.yaw)
        deviation_lat = params[ :(waypoints-2) ]
        offset = params[ (waypoints-2): ]

        for n in range(0, len(self.waypoints.yaw)-2):
            ax[n+1] -= deviation_lat[n]*np.sin(self.waypoints.yaw[n+1])
            ay[n+1] += deviation_lat[n]*np.cos(self.waypoints.yaw[n+1])

        bx, by, _, _ = self.quintic_bezier_path(ax, ay, offset)
        yaw, k = self.calc_yaw_curvature(bx, by)

        # cost of curvature continuity
        dk, _ = self.calc_d(k, k)
        absolute_dk = np.square(dk)
        continuity_cost = 70 * np.mean(absolute_dk)  

        # curvature cost
        absolute_k = np.square(k)
        curvature_cost = 1 * np.mean(absolute_k)

        return continuity_cost

    # Minimize quintic objective function
    def optimize_min_quintic(self):

        print("Attempting optimization minima")

        # create bounds for waypoint deviation and tangent length
        bnds = []
        for n in range(len(self.waypoints.yaw)-2):
            bnds.append((-self.bound, self.bound))
        for n in range(len(self.waypoints.yaw)):
            bnds.append((0, 1.3))
        bnds = tuple(bnds)

        # create initial guess for deviation and tangent lengths
        initial_guess = []
        for n in range(len(self.waypoints.yaw)-2):
            initial_guess.append(0.0)
        for n in range(len(self.waypoints.yaw)):
            initial_guess.append(0.2)

        result = minimize_parallel(self.quintic_objective_func, initial_guess, bounds=bnds)
        # result = optimize.minimize(self.quintic_objective_func, initial_guess, bounds=bnds)

        ax = self.waypoints.x.copy()
        ay = self.waypoints.y.copy()

        if result.success:
            print("optimized true")
            params = result.x
            
            # collects offsets for individual bezier curves
            waypoints = len(self.waypoints.yaw)
            deviation_lat = params[ :(waypoints-2) ]
            offset = params[ (waypoints - 2): ]
      

            # updated set of waypoints
            for n in range(0, len(self.waypoints.yaw)-2):
                ax[n+1] -= deviation_lat[n]*np.sin(self.waypoints.yaw[n+1])
                ay[n+1] += deviation_lat[n]*np.cos(self.waypoints.yaw[n+1])

            x, y, ctr_pt_x, ctr_pt_y = self.quintic_bezier_path(ax, ay, offset)
            yaw, k = self.calc_yaw_curvature(x, y)

            # update path optimized path and control points
            self.optimized_path = Path(x, y, yaw, k)
            self.ctr_points = Path(ctr_pt_x, ctr_pt_y, [], [])        

        else:
            print("optimization failure, defaulting")
            exit()



In [455]:
    # define input path
    ay = [0.0, 2.3, 6.25, 8.6, 8.2, 5.3, 2.6]
    ax = [0.0, 7.16, 13.68, 22.3, 30.64, 39.6, 50.4]
    boundary = 2.0

    %time spline = Spline(ax, ay, boundary)
    %time spline.optimize_min_quintic()

    # Path plot
    plt.subplots(1)
    plt.plot(spline.left_bound.x, spline.left_bound.y, '--r', alpha=0.5, label="left boundary")
    plt.plot(spline.right_bound.x, spline.right_bound.y, '--g', alpha=0.5, label="right boundary")
    plt.plot(spline.default_path.x, spline.default_path.y, '-y', label="default")
    plt.plot(spline.optimized_path.x, spline.optimized_path.y, '-m', label="optimized")
    plt.plot(spline.ctr_points.x, spline.ctr_points.y, 'xr', alpha=0.5, label="control points")
    plt.plot(spline.waypoints.x, spline.waypoints.y, '.', label="waypoints")
    plt.grid(True)
    plt.legend()
    plt.axis("equal")

    # Heading plot
    plt.subplots(1)
    plt.plot([np.rad2deg(iyaw) for iyaw in spline.default_path.yaw], "-y", label="original")
    plt.plot([np.rad2deg(iyaw) for iyaw in spline.optimized_path.yaw], "-m", label="optimized")
    plt.grid(True)
    plt.legend()
    plt.xlabel("line length[m]")
    plt.ylabel("yaw angle[deg]")

    # Curvature plot
    plt.subplots(1)
    plt.plot(spline.default_path.k, "-y", label="original")
    plt.plot(spline.optimized_path.k, "-m", label="optimized")
    plt.grid(True)
    plt.legend()
    plt.xlabel("line length[m]")
    plt.ylabel("curvature [1/m]")

    plt.show()

CPU times: user 70.2 ms, sys: 146 µs, total: 70.4 ms
Wall time: 69.7 ms
Attempting optimization minima
optimized true
CPU times: user 1.16 s, sys: 89 ms, total: 1.25 s
Wall time: 4.82 s


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …